<a href="https://colab.research.google.com/github/amruthakannan/QAS_using_gemini_LLM/blob/main/QAS_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U -q "google-generativeai>=0.8.3"

In [12]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.6 MB/s eta 0:00:

In [13]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display
import pandas as pd

In [3]:
GOOGLE_API_KEY = "API_CODE"

In [4]:
genai.configure(api_key= GOOGLE_API_KEY)

In [5]:
flash_model = genai.GenerativeModel('gemini-1.5-flash')
response = flash_model.generate_content("Explain LLM Agents to me in a simple and concise format")
print(response.text)

LLM agents are AI programs that use large language models (LLMs) to perform tasks autonomously.  Instead of just responding to prompts, they can plan, execute actions (like searching the web or accessing databases), and even reflect on their progress to achieve a goal.  Think of them as AI assistants with the ability to take initiative and complete complex instructions.



#Finding Embedding options

In [11]:
for m in genai.list_models():
  print(m.name)
  print(m.supported_generation_methods)

models/chat-bison-001
['generateMessage', 'countMessageTokens']
models/text-bison-001
['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001
['embedText', 'countTextTokens']
models/gemini-1.0-pro-latest
['generateContent', 'countTokens']
models/gemini-1.0-pro
['generateContent', 'countTokens']
models/gemini-pro
['generateContent', 'countTokens']
models/gemini-1.0-pro-001
['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.0-pro-vision-latest
['generateContent', 'countTokens']
models/gemini-pro-vision
['generateContent', 'countTokens']
models/gemini-1.5-pro-latest
['generateContent', 'countTokens']
models/gemini-1.5-pro-001
['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002
['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro
['generateContent', 'countTokens']
models/gemini-1.5-pro-exp-0801
['generateContent', 'countTokens']
models/gemini-1.5-pro-exp-0827
['generateContent', 'c

In [81]:
qas_dataset = pd.read_csv("/content/qas_dataset.csv")

In [82]:
qas_dataset.head()

,Paragraph,Questions,Answer
0,Construction is the process of constructing a ...,What is the process of constructing a building...,Construction
1,Construction is the process of constructing a ...,what is manufacturing?,manufacturing typically involves mass producti...
2,Many questions regarding prime numbers remain ...,What is the application of prime numbers used ...,public-key cryptography
3,"Starting in the late 1950s, American computer ...",What is Donald Davies credited with,Davies is credited with coining the modern nam...
4,"In the 1960s, a series of discoveries, the mos...",What is another word for the Earth's upper man...,asthenosphere


In [32]:
from chromadb import Documents, EmbeddingFunction, Embeddings, chromadb
from google.api_core import retry

class GeminiEmbeddingFn(EmbeddingFunction):

  document_type = 'document'


  def __call__(self, input: Documents) -> Embeddings:
    if self.document_type == 'document':
      embedding_task = 'retrieval_document'
    else:
      embedding_task = 'retrieval_query'

    retry_policy = {"retry": retry.Retry(preducate= retry.if_transient_error)}

    response = genai.embed_content(
        model = "models/text-embedding-004",
        content = input,
        task_type = embedding_task,
        request_options = retry_policy,

    )

    return response["embedding"]

In [37]:
paragraphs = list(qas_dataset.Paragraph.unique())

In [38]:
database_name = "squad_qas"
embed_function = GeminiEmbeddingFn()
embed_function.document_type = "document"
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name = database_name, embedding_function = embed_function)
db.add(documents = paragraphs, ids=[str(i) for i in range(len(paragraphs))])

In [40]:
db.count()

29

In [41]:
questions = list(qas_dataset.Questions.unique())

In [46]:
questions[0]

'What is the process of constructing a building or infrastructure?'

In [53]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")


In [67]:
embed_function.document_type = "query"
batch_prompts = []
for i in range(len(qas_dataset)):
  query = qas_dataset.Questions[i]
  [[retrieved_passage]] = db.query(query_texts= query, n_results = 1)["documents"]
  query= query.replace("\n", " ")
  retrieved_passage = retrieved_passage.replace("\n", " ")
  prompt = f""" You are a question answering bot. For the given passage and the questions
  below, find the answer. It can either be a single word or a complete sentence. The type of
  answer is dependent on the type of question (factoid or non-factoid).
  This is batch processed, so input only the answers in a new line each. So I can input
  it into a database

  PASSAGE: {retrieved_passage}
  QUESTION: {query}

  """
  batch_prompts.append(prompt)

batch_answers = model.generate_content(batch_prompts)

In [70]:
gemini_answers= [answer.text for answer in batch_answers]

In [73]:
gemini_answers[0].split("\n")[0]

'Construction'

In [83]:
for i in range(len(qas_dataset)):
   qas_dataset.loc[i, 'gemini_answer']= gemini_answers[0].split("\n")[i]


In [84]:
qas_dataset

,Paragraph,Questions,Answer,gemini_answer
0,Construction is the process of constructing a ...,What is the process of constructing a building...,Construction,Construction
1,Construction is the process of constructing a ...,what is manufacturing?,manufacturing typically involves mass producti...,Mass production of similar items without a des...
2,Many questions regarding prime numbers remain ...,What is the application of prime numbers used ...,public-key cryptography,Public-key cryptography
3,"Starting in the late 1950s, American computer ...",What is Donald Davies credited with,Davies is credited with coining the modern nam...,Coining the modern name packet switching
4,"In the 1960s, a series of discoveries, the mos...",What is another word for the Earth's upper man...,asthenosphere,Asthenosphere
5,The development of plate tectonics provided a ...,What is the area called where two plates move ...,divergent boundaries,The passage does not contain an answer to this...
6,The development of plate tectonics provided a ...,What is the area called where one plate subduc...,convergent boundaries,The passage does not contain an answer to this...
7,Seismologists can use the arrival times of sei...,What is the outer core called?,<No Answer>,Liquid
8,"During the mid-Eocene, it is believed that the...",What is the lake known as which was created by...,Solimões Basin,Solimões Basin
9,NASA's CALIPSO satellite has measured the amou...,What is the name of the satellite that measure...,CALIPSO,CALIPSO


In [87]:
qas_dataset.Paragraph[5]

"The development of plate tectonics provided a physical basis for many observations of the solid Earth. Long linear regions of geologic features could be explained as plate boundaries. Mid-ocean ridges, high regions on the seafloor where hydrothermal vents and volcanoes exist, were explained as divergent boundaries, where two plates move apart. Arcs of volcanoes and earthquakes were explained as convergent boundaries, where one plate subducts under another. Transform boundaries, such as the San Andreas fault system, resulted in widespread powerful earthquakes. Plate tectonics also provided a mechanism for Alfred Wegener's theory of continental drift, in which the continents move across the surface of the Earth over geologic time. They also provided a driving force for crustal deformation, and a new setting for the observations of structural geology. The power of the theory of plate tectonics lies in its ability to combine all of these observations into a single theory of how the lithos